In [10]:

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
warnings.filterwarnings('ignore')
import os
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import glob
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten, Dropout, Conv2D  ,MaxPooling2D, Reshape
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.optimizers import Adam
import tensorflow as tf

### Preparing Data

In [11]:
respeck_filepaths = glob.glob("../Respeck/*")
df1 = pd.DataFrame()
for rfp in respeck_filepaths:
    files = glob.glob(f"{rfp}/*")
    
    for file in files:
        [main_act,sub_act] = file.split(".csv")[0].split('_')[-2:]
        # main_activity = file.split(".csv")[0].split('_')[-2]
        
        df = pd.read_csv(file,index_col=0)
        df['activity'] = main_act
        df['sub_activity'] = sub_act
        df['user'] = rfp.split('\\')[-1]
        # print(df)
        df1 = pd.concat([df, df1], axis=0)

df1 = df1[df1['sub_activity'] == 'breathingNormal']     
df1.loc[df1['activity'].isin(('sitting', 'standing')),'activity'] = 'sitting_standing'

In [12]:
columns = ['user','activity','timestamp', 'accel_x', 'accel_y', 'accel_z']
# df1 = df1[columns]
df_har = df1[columns]
# removing null values
df_har = df_har.dropna()
df_har.shape
# transforming the user to float
df_har['user'] = df_har['user'].str.replace('s', '')
df_har['user'] = df_har['user'].apply(lambda x:int(x))


In [13]:
df_har.to_csv('task1.csv',index=False)

In [14]:
# ONLY RUN THIS AFTER CSV GENERATION
general_act_df = pd.read_csv('task1.csv')
general_act_df

,user,activity,timestamp,accel_x,accel_y,accel_z
0,98,sitting_standing,1697605965,-0.080811,-1.003235,0.052429
1,98,sitting_standing,1697606005,-0.069824,-0.988342,0.056091
2,98,sitting_standing,1697606045,-0.076416,-0.987122,0.047546
3,98,sitting_standing,1697606085,-0.082520,-0.985168,0.066345
4,98,sitting_standing,1697606125,-0.074951,-0.996887,0.034607
...,...,...,...,...,...,...
639844,1,ascending,1697636165,0.108887,-0.771790,-0.042053
639845,1,ascending,1697636205,0.157715,-0.820862,-0.046448
639846,1,ascending,1697636245,0.174072,-0.888733,-0.058167
639847,1,ascending,1697636285,0.071289,-0.782043,0.091492


### Segment Generation

In [15]:
random_seed = 42   
n_time_steps = 50 
n_features = 3 
step = 10
n_epochs = 1      
batch_size = 32

In [59]:
class CustomEncoder:
    def __init__(self):
        print('hello')
        self.class_mapping = {
            'sitting_standing': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            'lyingLeft':        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            'lyingRight':       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
            'lyingBack':        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            'lyingStomach':     [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
            'normalWalking':    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            'running':          [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
            'descending':       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
            'ascending':        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            'shuffleWalking':   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
            'miscMovement':     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        }

    def fit_transform(self, y):
        return np.array([self.class_mapping[cls] for cls in y])

    def inverse_transform(self, y):
        reverse_mapping = {v: k for k, v in self.class_mapping.items()}
        return np.array([reverse_mapping[val] for val in y])

In [40]:
def segments_no_overlap(data):
    segments = []
    labels = []

    for i in range(0,  data.shape[0]- n_time_steps, step):  

        xs = data['accel_x'].values[i: i + n_time_steps]

        ys = data['accel_y'].values[i: i + n_time_steps]

        zs = data['accel_z'].values[i: i + n_time_steps]

        label = stats.mode(data['activity'][i: i + n_time_steps])[0][0]

        segments.append([xs, ys, zs])

        labels.append(label)
        
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, n_time_steps, n_features)
    labels = np.asarray(labels).reshape(-1,1)
    
    # le = CustomLabelEncoder()
    # labels = le.fit_transform(labels)
    # return reshaped_segments,labels,le.class_mapping
    return reshaped_segments, labels
    # labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    # print(enc.categories_)
    

In [53]:
def get_segment_label(train_df,test_df):
    # saving training and testing data
    user = test_df['user'].unique()[0]
    train_df.to_csv(f'./t1_train_test_data/train/{user}_train.csv')
    test_df.to_csv(f'./t1_train_test_data/test/{user}_test.csv')
    
    # segmenting the data into windows 
    train_segments, train_labels = segments_no_overlap(train_df)
    test_segments, test_labels = segments_no_overlap(test_df)
    # transforming the labels into OneHotEncoding
    enc = OneHotEncoder(handle_unknown='ignore').fit(train_labels)
    train_labels_encoded = enc.transform(train_labels).toarray()
    test_labels_encoded = enc.transform(test_labels).toarray()
    
    return train_segments, train_labels_encoded, test_segments, test_labels_encoded, enc.categories_

### Model Training

In [8]:
def model_LSTM(X_train,y_train):
    model = Sequential()
    # RNN layer
    model.add(LSTM(units = 128, input_shape = (X_train.shape[1], X_train.shape[2])))
    # Dropout layer
    model.add(Dropout(0.5)) 
    # Dense layer with ReLu
    model.add(Dense(units = 64, activation='relu'))
    # Softmax layer
    model.add(Dense(y_train.shape[1], activation = 'softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    return model 

In [18]:
def model_cnn(trainX, trainy):
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(Reshape((n_timesteps, n_features, 1)))
    model.add(Conv2D(filters=64, kernel_size=(3,1), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3,1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=n_epochs, batch_size=batch_size, verbose=1)
    # evaluate model
    return model

### Driver Code

In [54]:
os.mkdir('./t1_train_test_data')
os.mkdir('./t1_train_test_data/train')
os.mkdir('./t1_train_test_data/test')
accuracies = {}

for user in general_act_df['user'].unique():
    if user != 91:
        continue
    
    train_df = general_act_df[general_act_df['user'] != user]
    test_df = general_act_df[general_act_df['user'] == user]
    
    # save_train_test_data(user,train_df,test_df)

    X_train, y_train, X_test, y_test, categories = get_segment_label(train_df,test_df)
    
    # model = model_LSTM(X_train,y_train)
    # history = model.fit(X_train, y_train, epochs = n_epochs, validation_split = 0.20, batch_size = batch_size, verbose = 1)
    
    model  = model_cnn(X_train,y_train)
    
    loss, accuracy = model.evaluate(X_test, y_test, batch_size = batch_size, verbose = 1)
    print(f"Test Accuracy ({user}):", accuracy)
    print(f"Test Loss ({user}):", loss)
    
    accuracies[user] = {'loss':loss, 'accuracy': accuracy}
    # break
    

22/22 [==============================] - 0s 4ms/step - loss: 0.4617 - accuracy: 0.8341
Test Accuracy (91): 0.834074079990387
Test Loss (91): 0.4616837203502655


In [56]:
y_test

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [54]:
a_df = pd.DataFrame(accuracies)
a_df.to_csv('./t1_train_test_data/t1_loo_accuracies.csv')

In [90]:
import tensorflow_model_optimization as tfmot

In [91]:
quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_4 (QuantizeL  (None, 50, 3)            3         
 ayer)                                                           
                                                                 
 quant_reshape_7 (QuantizeWr  (None, 50, 3, 1)         1         
 apperV2)                                                        
                                                                 
 quant_conv2d_14 (QuantizeWr  (None, 48, 3, 64)        387       
 apperV2)                                                        
                                                                 
 quant_conv2d_15 (QuantizeWr  (None, 46, 3, 64)        12483     
 apperV2)                                                        
                                                                 
 quant_dropout_11 (QuantizeW  (None, 46, 3, 64)      

In [95]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.target_spec.supported_types = [tf.float16]
quantized_tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmpruc2c2yz\assets


INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmpruc2c2yz\assets


In [101]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open(f'../models/cnn_model_task{1}_{n_time_steps}_{n_features}.tflite', 'wb') as f:
  f.write(quantized_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmp7vr0w4hs\assets


INFO:tensorflow:Assets written to: C:\Users\chhal\AppData\Local\Temp\tmp7vr0w4hs\assets


In [56]:
interpreter = tf.lite.Interpreter(model_path="../models/cnn_model_task1_50_3.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on one instance
interpreter.set_tensor(input_details[0]['index'], np.float32(X_test[0:1]))
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.00304507 0.00304507 0.00710347 0.00780451 0.00304507 0.05904093
  0.00304507 0.00304507 0.00304507 0.00304507 0.90473557]]


In [16]:
correct_predictions = 0
total_predictions = 0

for i in range(len(X_test)):
    interpreter.set_tensor(input_details[0]['index'], np.float32(X_test[i:i+1]))
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    total_predictions += 1
    if np.argmax(output_data) == np.argmax(y_test[i]):
        correct_predictions += 1

accuracy = correct_predictions / total_predictions
print('Test accuracy:', accuracy)

Test accuracy: 0.16427783902976847


In [4]:
t91 = pd.read_csv('./t1_data/test/91_test.csv')
print(t91['activity'].unique())

['sitting_standing' 'normalWalking' 'miscMovement' 'lyingStomach'
 'lyingRight' 'lyingLeft' 'lyingBack' 'ascending']
